In [1]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-e35vhzjm
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-e35vhzjm
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.7 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369490 sha256=a1a863895ade09333a5017315d2cece11db283df649523f114222eb74ba501ac
  Stored in directory: /tmp/pip-ephem-wheel-cache-x65w67dh/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [2]:
pip install -U albumentations

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.9/227.9 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 606.4/606.4 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.7/291.7 kB 13.4 MB/s eta 0:00:00
  Attempting uninstall: albucore
    Found existing installation: albucore 0.0.17
    Uninstalling albucore-0.0.17:
      Successfully uninstalled albucore-0.0.17
  Attempting uninstall: albumentations
    Found existing installation: albumentations 1.4.17
    Uninstalling albumentations-1.4.17:
      Successfully uninstalled albumentations-1.4.17
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import gc
import sys
from PIL import Image
import cv2
import math, random
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, GroupKFold, StratifiedKFold

from collections import OrderedDict

import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW

import timm
from transformers import get_cosine_schedule_with_warmup

import albumentations as A
from sklearn.preprocessing import LabelEncoder
import clip

from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import DataLoader
import wandb
from tqdm import tqdm

In [4]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
DEBUG = False
N_WORKERS = os.cpu_count() 
USE_AMP = False # can change True if using T4 or newer than Ampere
SEED = 42
CLIP_MODEL_NAME = "RN101"
IMG_SIZE = [224, 224]
AUG_PROB = 0.2
BATCH_SIZE = 64
EPOCHS = 10
if DEBUG:
    EPOCHS = 3
GRAD_ACC = 1
N_FOLDS = 5
MAX_GRAD_NORM = None
EARLY_STOPPING_EPOCH = 3
OUTPUT_DIR = f'clip_landmark_results'

In [5]:
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [6]:
train_df = pd.read_csv("/kaggle/input/google-landmarks-v2-index-subset-metadata/cleaned_landmark_index_subset_150k_v2.csv")
if DEBUG:
    # Reduce to 1000 samples for quick debugging
    train_df = train_df.sample(1000, random_state=42).reset_index(drop=True)
train_df.head()

,id,landmark_id,category,supercategory,hierarchical_label,natural_or_human_made,cleaned_supercategory,image_path,landmark_name
0,73792c1235ff2625,33618,http://commons.wikimedia.org/wiki/Category:Don...,sculpture,sculpture,human-made,sculpture,/kaggle/input/google-landmarks-v2-index-set-p3...,Dona i Ocell
1,a3751bcb2740c891,33618,http://commons.wikimedia.org/wiki/Category:Don...,sculpture,sculpture,human-made,sculpture,/kaggle/input/google-landmarks-v2-index-set-p4...,Dona i Ocell
2,2f32b63b86113c22,33618,http://commons.wikimedia.org/wiki/Category:Don...,sculpture,sculpture,human-made,sculpture,/kaggle/input/google-landmarks-v2-index-set-p1...,Dona i Ocell
3,657c10997e4f399c,33618,http://commons.wikimedia.org/wiki/Category:Don...,sculpture,sculpture,human-made,sculpture,/kaggle/input/google-landmarks-v2-index-set-p2...,Dona i Ocell
4,029ba12e494a33ff,33618,http://commons.wikimedia.org/wiki/Category:Don...,sculpture,sculpture,human-made,sculpture,/kaggle/input/google-landmarks-v2-index-set-p1...,Dona i Ocell


In [7]:
N_CLASSES = train_df.landmark_name.nunique()
class_names = train_df.landmark_name.unique()

In [8]:
# Load CLIP model
embedding_device = "cpu"
model, _ = clip.load(CLIP_MODEL_NAME, device=embedding_device)

# Define class names and tokenize
"""text_prompts = [f"A photo of {name} or its part" for name in class_names]
text_tokens = clip.tokenize(text_prompts).to(embedding_device)

# Compute text embeddings
with torch.no_grad():
    text_embeddings = model.encode_text(text_tokens)
text_embeddings = text_embeddings / text_embeddings.norm(dim=-1, keepdim=True)"""  # Normalize for cosine similarity



100%|███████████████████████████████████████| 278M/278M [00:11<00:00, 24.5MiB/s]


'text_prompts = [f"A photo of {name} or its part" for name in class_names]\ntext_tokens = clip.tokenize(text_prompts).to(embedding_device)\n\n# Compute text embeddings\nwith torch.no_grad():\n    text_embeddings = model.encode_text(text_tokens)\ntext_embeddings = text_embeddings / text_embeddings.norm(dim=-1, keepdim=True)'

In [9]:
text_embeddings = torch.load("/kaggle/input/text_embeddings/pytorch/default/1/text_embeddings.pt", weights_only=True)

In [10]:
class_name_to_idx = {name: idx for idx, name in enumerate(class_names)}
idx_to_class_name = {idx: class_name for class_name, idx in class_name_to_idx.items()}

In [11]:
transforms_train = A.Compose([
    A.RandomBrightnessContrast(brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), p=AUG_PROB),
    A.OneOf([
        A.MotionBlur(blur_limit=5),
        A.MedianBlur(blur_limit=5),
        A.GaussianBlur(blur_limit=5),
        A.GaussNoise(var_limit=(5.0, 30.0)),
    ], p=AUG_PROB),

    A.OneOf([
        A.OpticalDistortion(distort_limit=1.0),
        A.GridDistortion(num_steps=5, distort_limit=1.),
        A.ElasticTransform(alpha=3),
    ], p=AUG_PROB),

    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=AUG_PROB),
    A.Resize(IMG_SIZE[0], IMG_SIZE[1]),
    A.CoarseDropout(max_holes=8, max_height=32, max_width=32, min_holes=1, min_height=8, min_width=8, p=AUG_PROB),    
    A.Normalize(mean=[0.48145466, 0.4578275, 0.40821073], 
                         std=[0.26862954, 0.26130258, 0.27577711])
])

transforms_val = A.Compose([
    A.Resize(IMG_SIZE[0], IMG_SIZE[1]),
    A.Normalize(mean=[0.48145466, 0.4578275, 0.40821073], 
                         std=[0.26862954, 0.26130258, 0.27577711])
])

/opt/conda/lib/python3.10/site-packages/pydantic/main.py:212: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


In [12]:
class GoogleLandmarksDataset(Dataset):
    def __init__(self, df, class_name_to_idx, transform=None):
        self.df = df
        self.class_name_to_idx = class_name_to_idx  # Maps class name to index
        self.transform = transform
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        t = self.df.iloc[idx]
        class_name = t['landmark_name']
        class_name_idx = self.class_name_to_idx[class_name]
        
        p = f'{t["image_path"]}'
        img = Image.open(p)
        img = np.array(img)
            
        # Apply transformations if provided
        if self.transform:
            img = self.transform(image=np.array(img))['image']
        
        # Convert image to tensor if needed
        if not isinstance(img, torch.Tensor):
            img = torch.tensor(img.transpose(2, 0, 1), dtype=torch.float32)
        
        return img, class_name_idx


In [13]:
class CrossModalDistillationModel(nn.Module):
    def __init__(self, teacher_model, student_model, text_embeddings, projection_dim=512):
        super().__init__()
        self.teacher = teacher_model
        self.student = student_model
        self.text_embeddings = text_embeddings
        
        # Freeze teacher
        for param in self.teacher.parameters():
            param.requires_grad = False
        
        # Get input features dimension
        in_features = student_model.get_classifier().in_features
        
        # Single projection head for student
        self.projection_head = nn.Sequential(
            nn.Linear(in_features, 1024),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(1024, projection_dim),
            nn.BatchNorm1d(projection_dim)  # Added BatchNorm for stable training
        )
        
        # Classification head
        self.classifier = nn.Linear(in_features, student_model.get_classifier().out_features)
        
        # Temperature parameter
        self.temperature = nn.Parameter(torch.ones([]) * np.log(1 / 0.07))
        
    def forward(self, images, return_features=True):
        batch_size = images.size(0)
        
        # Teacher forward pass
        with torch.no_grad():
            teacher_features = self.teacher.encode_image(images)
            teacher_features = F.normalize(teacher_features, dim=-1)
            teacher_features = teacher_features.float()
            self.text_embeddings = self.text_embeddings.float()
            teacher_logits = torch.matmul(teacher_features, self.text_embeddings.t())
            teacher_logits = teacher_logits * self.temperature.exp()

        # Student forward pass
        student_features = self.student.forward_features(images)
        student_features = F.adaptive_avg_pool2d(student_features, (1, 1))
        student_features = student_features.view(batch_size, -1)

        # Project student features
        student_projected = self.projection_head(student_features)
        student_projected = F.normalize(student_projected, dim=-1)

        # Get classification logits
        student_logits = self.classifier(student_features)

        if return_features:
            return {
                'teacher_logits': teacher_logits,
                'student_logits': student_logits,
                'teacher_features': teacher_features,
                'student_projected': student_projected,
                'student_features': student_features
            }
        
        return student_logits

In [14]:
class Config:
    def __init__(self):
        self.alpha = 0.2
        self.beta = 0.2
        self.gamma = 0.5
        self.temperature = 2.0
        self.device = device
        self.batch_size = BATCH_SIZE
        self.n_workers = N_WORKERS
        self.max_grad_norm = MAX_GRAD_NORM
        self.log_interval = 100
        self.use_amp = False
        self.use_wandb = False  # Set to True if using wandb
        self.warmup_steps = 100
        self.gradient_accumulation_steps = 1

In [15]:

class EnhancedDistillationLoss(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.alpha = config.alpha  # KD loss weight
        self.beta = config.beta    # Feature alignment weight
        self.gamma = config.gamma  # CE loss weight
        self.T = config.temperature
        self.contrastive_weight = 0.1  # Weight for contrastive loss
        
    def contrastive_loss(self, features, labels):
        # Implement contrastive loss between positive pairs
        similarity = torch.matmul(features, features.t())
        mask = (labels.unsqueeze(0) == labels.unsqueeze(1)).float()
        
        # Temperature-scaled similarities
        similarity = similarity / self.T
        
        # Positive and negative pairs
        exp_sim = torch.exp(similarity)
        pos_sim = exp_sim * mask
        neg_sim = exp_sim * (1 - mask)
        
        loss = -torch.log(
            pos_sim.sum(1) / (pos_sim.sum(1) + neg_sim.sum(1))
        ).mean()
        
        return loss

    def forward(self, outputs, labels):
        teacher_logits = outputs['teacher_logits']
        student_logits = outputs['student_logits']
        teacher_features = outputs['teacher_features']
        student_projected = outputs['student_projected']
        
        # 1. Knowledge Distillation Loss
        kd_loss = F.kl_div(
            F.log_softmax(student_logits / self.T, dim=1),
            F.softmax(teacher_logits / self.T, dim=1),
            reduction="batchmean"
        ) * (self.T ** 2)
        
        # 2. Feature Alignment Loss
        feature_loss = F.mse_loss(student_projected, teacher_features)
        
        # 3. Cross Entropy Loss
        ce_loss = F.cross_entropy(student_logits, labels)
        
        # 4. Contrastive Loss
        cont_loss = self.contrastive_loss(student_projected, labels)
        
        total_loss = (
            self.alpha * kd_loss +
            self.beta * feature_loss +
            self.gamma * ce_loss +
            self.contrastive_weight * cont_loss
        )
        
        return total_loss, {
            'kd_loss': kd_loss.item(),
            'feature_loss': feature_loss.item(),
            'ce_loss': ce_loss.item(),
            'cont_loss': cont_loss.item()
        }

In [16]:
def train_epoch(model, train_loader, criterion, optimizer, config, scheduler=None):
    model.train()
    total_loss = 0
    metrics = defaultdict(float)  # Tracks additional metrics
    scaler = GradScaler() if config.use_amp else None
    
    pbar = tqdm(train_loader, desc='Training')
    
    for batch_idx, (images, labels) in enumerate(pbar):
        images, labels = images.to(config.device), labels.to(config.device)
        
        # Mixed precision training
        if config.use_amp:
            with autocast():
                outputs = model(images, return_features=True)
                loss, batch_metrics = criterion(outputs, labels)
            scaler.scale(loss).backward()
            
            if config.max_grad_norm:
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), config.max_grad_norm)
            
            scaler.step(optimizer)
            scaler.update()
        else:
            outputs = model(images, return_features=True)
            loss, batch_metrics = criterion(outputs, labels)
            loss.backward()
            
            if config.max_grad_norm:
                torch.nn.utils.clip_grad_norm_(model.parameters(), config.max_grad_norm)
            
            optimizer.step()
        
        optimizer.zero_grad()
        if scheduler:
            scheduler.step()
        
        # Update loss and metrics
        total_loss += loss.item()
        for k, v in batch_metrics.items():
            metrics[k] += v
        
        # Update progress bar with detailed information
        pbar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'avg_loss': f'{total_loss / (batch_idx + 1):.4f}',
            'lr': scheduler.get_last_lr()[0] if scheduler else optimizer.param_groups[0]['lr']
        })
    
    # Normalize metrics over all batches
    metrics = {k: v / len(train_loader) for k, v in metrics.items()}
    return total_loss / len(train_loader), metrics

In [17]:
def compute_topk_accuracy(outputs, labels, k):
    """
    Computes the top-k accuracy for the given outputs and labels.
    
    Args:
        outputs (torch.Tensor or dict): Model outputs, expected as a tensor.
        labels (torch.Tensor): Ground truth labels.
        k (int): The 'k' in top-k accuracy.
    
    Returns:
        float: Top-k accuracy as a percentage.
    """
    if isinstance(outputs, dict):
        outputs = outputs['student_logits']  # Update the key to match your dictionary structure
    
    _, top_k_preds = outputs.topk(k, dim=1)
    top_k_correct = top_k_preds.eq(labels.view(-1, 1).expand_as(top_k_preds))
    return top_k_correct.any(dim=1).float().mean().item()

In [18]:
def validate_epoch(model, val_loader, criterion, config):
    model.eval()
    total_loss = 0
    metrics = defaultdict(float)
    
    pbar = tqdm(val_loader, desc='Validating')
    
    with torch.no_grad():  # Disable gradient computation
        for batch_idx, (images, labels) in enumerate(pbar):
            images, labels = images.to(config.device), labels.to(config.device)
            
            # Forward pass
            outputs = model(images, return_features=True)
            loss, batch_metrics = criterion(outputs, labels)
            
            # Accumulate loss and metrics
            total_loss += loss.item()
            metrics['top1_acc'] += compute_topk_accuracy(outputs, labels, 1)
            metrics['top5_acc'] += compute_topk_accuracy(outputs, labels, 5)
            metrics['top10_acc'] += compute_topk_accuracy(outputs, labels, 10)
            
            # Update progress bar
            pbar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'avg_loss': f'{total_loss / (batch_idx + 1):.4f}'
            })
    
    # Normalize metrics over all batches
    metrics = {k: v / len(val_loader) for k, v in metrics.items()}
    return total_loss / len(val_loader), metrics

In [19]:
from collections import defaultdict

In [20]:
train_ds = GoogleLandmarksDataset(train_df, class_name_to_idx=class_name_to_idx, transform=transforms_train)
train_dl = DataLoader(
    train_ds,
    batch_size=BATCH_SIZE,
    shuffle=True,
    pin_memory=True,
    drop_last=True,
    num_workers=N_WORKERS
)


In [21]:
teacher_model, preprocess = clip.load(CLIP_MODEL_NAME, device=device)
student_model = timm.create_model("resnet18", pretrained=True, num_classes=N_CLASSES)

# Initialize the improved distillation model
distillation_model = CrossModalDistillationModel(
    teacher_model=teacher_model,
    student_model=student_model,
    text_embeddings=text_embeddings.to(device),
    projection_dim=512  # Same dimension as CLIP's feature space
)
distillation_model.to(device)

# Optimizer with weight decay
optimizer = AdamW([
    {'params': distillation_model.student.parameters(), 'lr': 1e-4},
    {'params': distillation_model.projection_head.parameters(), 'lr': 1e-4}
], weight_decay=0.01)

# Scheduler with warmup
num_training_steps = len(train_dl) * EPOCHS
num_warmup_steps = min(1000, num_training_steps // 20)  # 10% of training steps or 1000, whichever is smaller
scheduler = get_cosine_schedule_with_warmup(
    optimizer, 
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps
)

# Training loop with validation
patience = EARLY_STOPPING_EPOCH
patience_counter = 0

scaler = GradScaler() if USE_AMP else None

sfk = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
training_folds = [0] 
fold_metrics = []

for fold, (trn_idx, val_idx) in enumerate(sfk.split(train_df, train_df.landmark_id.tolist())):
    if fold not in training_folds:
        continue

    print('#'*30)
    print(f'Starting Fold {fold}')
    print('#'*30)

    df_train = train_df.iloc[trn_idx]
    df_valid = train_df.iloc[val_idx]

    # Initialize Datasets and DataLoaders
    train_ds = GoogleLandmarksDataset(df_train, class_name_to_idx=class_name_to_idx, transform=transforms_train)
    valid_ds = GoogleLandmarksDataset(df_valid, class_name_to_idx=class_name_to_idx, transform=transforms_val)
    
    train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True, drop_last=True, num_workers=N_WORKERS)
    valid_dl = DataLoader(valid_ds, batch_size=BATCH_SIZE*2, shuffle=False, pin_memory=True, drop_last=False, num_workers=N_WORKERS)
    
    best_val_loss = float('inf')
    patience_counter = 0

    # Training Loop
    for epoch in range(EPOCHS):
        train_loss, train_metrics = train_epoch(
            model=distillation_model,
            train_loader=train_dl,
            criterion=EnhancedDistillationLoss(Config()),
            optimizer=optimizer,
            config=Config(),
            scheduler=scheduler
        )

        val_loss, val_metrics = validate_epoch(
            model=distillation_model,
            val_loader=valid_dl,
            criterion=EnhancedDistillationLoss(Config()),
            config=Config()
        )

        # Save Best Model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save({
                'fold': fold,
                'epoch': epoch,
                'model_state_dict': distillation_model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_loss': val_loss,
            }, f'{OUTPUT_DIR}/best_model_fold{fold}.pth')
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f'Early stopping triggered at epoch {epoch+1}')
                break
    
    fold_metrics.append({
        'fold': fold,
        'train_loss': train_loss,
        'val_loss': val_loss,
        'top1_acc': val_metrics['top1_acc'],
        'top5_acc': val_metrics['top5_acc'],
        'top10_acc': val_metrics['top10_acc'],
    })

# Print Summary
for metric in fold_metrics:
    print(metric)

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

##############################
Starting Fold 0
##############################


Validating: 100%|██████████| 235/235 [01:47<00:00,  2.19it/s, loss=4.4525, avg_loss=4.3389]


{'fold': 0, 'train_loss': 4.420293859227498, 'val_loss': 4.338883466543035, 'top1_acc': 0.09564494680851064, 'top5_acc': 0.14148936170212767, 'top10_acc': 0.1540226063829787}
